In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
from pathlib import Path
import random

import ultralytics as ul
from ruamel.yaml import YAML

import bb
import tt

LOG = logging.getLogger(__name__)
tt.logging_init()

# Settings

In [3]:
data_path = Path.home() / "src/data"
mc_data_path = data_path / "minecraft/info.json"
yolo_path = data_path / "yolo"
models_path = yolo_path / "models"

ul.settings.update(
    {
        "datasets_dir": str(yolo_path / "datasets"),
        "weights_dir": str(yolo_path / "weights"),
        "runs_dir": str(yolo_path / "runs"),
    }
)

SEED=325
random.seed(SEED)

# Export data to yolo

In [4]:
# rm -rf data.yaml images labels
dset = bb.Dataset.load(mc_data_path)
dset.to_yolo(yolo_path, 80)

[20:17:07]:[INFO]:(bb): Exported 408 images to YOLO format at /Users/joe/src/data/yolo


In [5]:
dset = bb.Dataset.load(mc_data_path)
display(dset.dataset_stats())
display(dset.category_stats())

num_images                    389
num_bboxes                    503
avg_bboxes_per_image     1.293059
num_categories                  9
most_common_category          cow
least_common_category       sheep
avg_bbox_width           0.215928
avg_bbox_height          0.297827
avg_bbox_area            0.085275
min_bbox_area             0.00176
max_bbox_area            0.790986
dtype: object

,count,avg_width,avg_height,avg_area
category,,,,
cow,64,0.2150,0.2616,0.0776
chicken,62,0.1495,0.2031,0.0374
pig,62,0.2270,0.2130,0.0658
creeper,58,0.1819,0.3474,0.0808
spider,57,0.3913,0.3022,0.1609
zombie,55,0.1896,0.3156,0.0778
enderman,50,0.2002,0.4796,0.1179
skeleton,48,0.1699,0.3593,0.0899
sheep,47,0.2142,0.2407,0.0677


# Train

In [6]:
model = ul.YOLO(models_path / "yolo11n.pt")

In [7]:
train_args_yaml = f"""
# https://docs.ultralytics.com/usage/cfg/
data: {yolo_path / "data.yaml"}
epochs: 200
imgsz: 640
device: mps
seed: {SEED}
save_period: 20
patience: 0
# cache: true

# Optimizer
optimizer: AdamW
lr0: 0.0008             # close to auto's 7.69e-4
lrf: 0.01               # default
momentum: 0.9           # default, AdamW uses betas; Ultralytics maps momentum-ish
weight_decay: 0.0005    # default

# Augmentation https://docs.ultralytics.com/guides/yolo-data-augmentation/

auto_augment: null
# Color augmentations
hsv_h: 0.01          # Reduce from 0.015 - color is discriminative for mobs
hsv_s: 0.7           # Keep - lighting varies in game
hsv_v: 0.4           # Keep - day/night/caves matter

# Geometric augmentations
degrees: 5.0         # rotation
translate: 0.1       # Keep
scale: 0.5           # Keep - distance variation
shear: 0.0           # Keep off
perspective: 0.0001  # Add tiny bit - camera perspective
flipud: 0.0          # Keep off
fliplr: 0.5          # Keep

# Advanced augmentations - BE CAREFUL WITH SMALL DATASET
mosaic: 0.0
mixup: 0.0
cutmix: 0.0
copy_paste: 0.0
erasing: 0.2

# name
# optimizer
# freeze
# multi_scale
# cls, box
"""

In [8]:
yaml = YAML(typ="safe")
train_args = yaml.load(train_args_yaml)

In [9]:
import pprint
pprint.pp(train_args)

{'data': '/Users/joe/src/data/yolo/data.yaml',
 'epochs': 200,
 'imgsz': 640,
 'device': 'mps',
 'seed': 325,
 'save_period': 20,
 'patience': 0,
 'optimizer': 'AdamW',
 'lr0': 0.0008,
 'lrf': 0.01,
 'momentum': 0.9,
 'weight_decay': 0.0005,
 'auto_augment': None,
 'hsv_h': 0.01,
 'hsv_s': 0.7,
 'hsv_v': 0.4,
 'degrees': 5.0,
 'translate': 0.1,
 'scale': 0.5,
 'shear': 0.0,
 'perspective': 0.0001,
 'flipud': 0.0,
 'fliplr': 0.5,
 'mosaic': 0.0,
 'mixup': 0.0,
 'cutmix': 0.0,
 'copy_paste': 0.0,
 'erasing': 0.2}


In [10]:
train_results = model.train(**train_args)

New https://pypi.org/project/ultralytics/8.3.240 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.223 🚀 Python-3.13.9 torch-2.9.0 MPS (Apple M3 Max)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=None, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/joe/src/data/yolo/data.yaml, degrees=5.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.2, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0008, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/Users/joe/src/data/yolo/models/yolo11n.pt, momentum=0.9, mosaic=0.0, multi_scale=False, name=train, nbs=64, nms=Fal